In [1]:
import geopandas
import matplotlib.pyplot as plt
import overpass
import numpy as np
import pandas as pd

In [2]:
def make_feature_dict(df, features=None, prefix=None):
    """
    
    Parameters
    ----------
    df : geopandas.geodataframe.GeoDataFrame
        Dataframe containing features pulled from overpass API
        
    features : list
        List of column names associated with high-level feature names
        employed by overpass/OSM. Reference here:
        https://wiki.openstreetmap.org/wiki/Map_Features
        
    prefix : str
        Additional identifier placed before feature name.
        
    Returns
    -------
    dict
        Nested dictionary of features with value counts as eventual values.
        
    """
    
    if prefix!=None:
        pass
    else:
        prefix=''
    
    if features==None:
        features = df.columns[1:]
        
    feature_dict = {}
    for feature in features:
        if feature in df.columns:
            series = df[feature].value_counts()
            feature_dict[prefix+feature] = { k:v for (k,v) in zip(series.index, series.values)}
        else:
            feature_dict[prefix+feature] = None
    return feature_dict

In [3]:
df_ct_collisions = pd.read_csv("vt_deer_collisions_2010_2019_190919.csv", index_col=0)

In [4]:
endpoint = 'https://overpass.kumi.systems/api/interpreter'
endpoint = 'https://z.overpass-api.de/api/interpreter'
endpoint = 'https://lz4.overpass-api.de/api/interpreter'
api_pass = overpass.API(timeout=7200, endpoint=endpoint)

In [6]:
#road_responses, area_nodes, area_ways, area_rels = [], [], [], []

n=403
for i in df_ct_collisions.Coordinates[n:]:
    print(n, end=', ')
    if type(i) == str:
        lat, lon = [float(coord) for coord in i.split(',')]
        
        road_query = 'way(around:1, {}, {});(._;>;);out;'.format(lat, lon)
        # Getting overpass road responses
        road_response = api_pass.get(road_query)
        #road_responses.append(road_response)
        road_geodf = geopandas.GeoDataFrame.from_features(road_response)
        culled_road = road_geodf.dropna(axis='index', how='all', subset=road_geodf.columns[1:])
        # Overwriting feature_dict each time
        feature_dict = make_feature_dict(culled_road, ['highway', 'surface'], prefix='road_')
        #time.sleep(1)
    
        area_node_query = 'node(around:500, {}, {});(._;>;);out;'.format(lat, lon)
        # Getting overpass area responses
        response = api_pass.get(area_node_query)
        #area_nodes.append(response)
        geodf = geopandas.GeoDataFrame.from_features(response)
        culled_geodf = geodf.dropna(axis='index', how='all', subset=geodf.columns[1:])
        # Update dictionary
        feature_dict.update(make_feature_dict(culled_geodf, prefix='area_node_'))
                
        area_way_query = 'way(around:500, {}, {});(._;>;);out;'.format(lat, lon)
        # Getting overpass area responses
        response = api_pass.get(area_way_query)
        #area_ways.append(response)
        geodf = geopandas.GeoDataFrame.from_features(response)
        culled_geodf = geodf.dropna(axis='index', how='all', subset=geodf.columns[1:])
        # Update dictionary
        feature_dict.update(make_feature_dict(culled_geodf, prefix='area_way_'))

        area_rel_query = 'rel(around:500, {}, {});(._;>;);out;'.format(lat, lon)
        # Getting overpass area responses
        response = api_pass.get(area_rel_query)
        #area_rels.append(response)
        geodf = geopandas.GeoDataFrame.from_features(response)
        culled_geodf = geodf.dropna(axis='index', how='all', subset=geodf.columns[1:])
        # Update dictionary
        feature_dict.update(make_feature_dict(culled_geodf, prefix='area_rel_'))
        
        #time.sleep(1)

        # Write the file
        pd.io.json.json_normalize(feature_dict).to_csv('overpass_point_data/{}.csv'.format(str(n).rjust(4,'0')))
        n += 1
    else:
        pass
    #print(point_query)

403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 

In [23]:
feature_dict

{'road_highway': {'residential': 2}, 'road_surface': None}

In [32]:
culled_geodf

,geometry,ele,gnis:id,is_in,name,place,population,wikidata,amenity,gnis:county_id,...,gnis:feature_id,gnis:state_id,religion,addr:state,gnis:county_name,gnis:import_uuid,gnis:reviewed,source,leisure,office
0,POINT (-72.482986 43.298698),122,1459648,"Windsor,VT,USA",Springfield,town,8792,Q40212,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,POINT (-72.488119 43.295505),171,NaN,NaN,Union Street School,NaN,NaN,NaN,school,027,...,1459993,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,POINT (-72.486047 43.297772),152,NaN,NaN,Holy Trinity Othodox Church,NaN,NaN,NaN,place_of_worship,027,...,1922407,50,christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,POINT (-72.48512599999999 43.298902),146,NaN,NaN,Park Street School,NaN,NaN,NaN,school,027,...,1922522,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,POINT (-72.48177200000001 43.299773),131,NaN,NaN,Springfield United Methodist Church,NaN,NaN,NaN,place_of_worship,027,...,1922624,50,christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
299,POINT (-72.48512599999999 43.298902),146,NaN,NaN,Springfield Supervisory Union,NaN,NaN,NaN,school,027,...,2427514,50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,POINT (-72.48141800000001 43.296903),132,NaN,NaN,Springfield Town Offices,NaN,NaN,NaN,public_building,NaN,...,1922623,NaN,NaN,VT,Windsor,57871b70-0100-4405-bb30-88b2e001a944,no,USGS Geonames,NaN,NaN
431,POINT (-72.482128 43.297541),NaN,NaN,NaN,First Congregational Church,NaN,NaN,NaN,place_of_worship,NaN,...,NaN,NaN,christian,NaN,NaN,NaN,NaN,NaN,NaN,NaN
432,POINT (-72.48260000000001 43.29977),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,park,NaN
437,POINT (-72.48468099999999 43.297594),NaN,NaN,NaN,Department of Motor Vehicles,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,government


```python
endpoint = 'https://overpass.kumi.systems/api/interpreter'
endpoint = 'https://z.overpass-api.de/api/interpreter'
endpoint = 'https://lz4.overpass-api.de/api/interpreter'
api_pass = overpass.API(timeout=7200, endpoint=endpoint)
road_responses, area_responses = list(), list()
i = 0

for index, row in collision_per_aadt.iloc[:,:].iterrows(): 
    print(i, end=', ')
    poly = row.geometry.buffer(.0025, cap_style=3)
    test_shape = poly.simplify(0.001, preserve_topology=False)
    
    # Building overpass queries
    road_query = 'nwr(around:1, {c[0]}, {c[1]});(._;>;);out;'.format(
        c=row.geometry.interpolate(0.5, normalized = True).coords[:][0])
    area_query = 'nwr(poly:"{}");out;'.format(
        ' '.join('{c[0]} {c[1]}'.format(c=c) for c in test_shape.exterior.coords[:]))
    
    # Getting overpass road responses
    road_response = api_pass.get(road_query)
    road_responses.append(road_response)
    road_geodf = geopandas.GeoDataFrame.from_features(road_response)
    culled_road = road_geodf.dropna(axis='index', how='all', subset=road_geodf.columns[1:])
    # Overwriting feature_dict each time
    feature_dict = make_feature_dict(culled_road, ['highway', 'surface'], prefix='road_')
    time.sleep(5)
    
    # Getting overpass area responses
    response = api_pass.get(area_query)
    area_responses.append(response)
    geodf = geopandas.GeoDataFrame.from_features(response)
    culled_geodf = geodf.dropna(axis='index', how='all', subset=geodf.columns[1:])
    feature_dict.update(make_feature_dict(culled_geodf, cols, prefix='area_'))
    pd.io.json.json_normalize(feature_dict).to_csv('overpass_data/{}.csv'.format(str(i).rjust(4,'0')))
    time.sleep(5)
    i += 1
```